In [87]:
from operator import itemgetter

import numpy as np
import pandas as pd
from more_itertools import flatten
from sklearn.metrics import classification_report
from sklearn.utils import gen_batches, shuffle
from tqdm.notebook import tqdm

from nya_ml.models.blanchefort_rubert_sentiment import BlanchefortRuBertSentiment
from nya_ml.models.tatyana_rubert_sentiment import TatyanaRuBertSentiment
from nya_ml_research.config import DATA_PATH

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
blanchefort = BlanchefortRuBertSentiment.load()

In [7]:
tatyana = TatyanaRuBertSentiment.load()

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/943 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [8]:
blanchefort.predict(['Привет всем любителям пончиков'])

{'neutral': 0.9178354144096375,
 'positive': 0.08072204142808914,
 'negative': 0.0014424966648221016}

In [12]:
tatyana.predict(['Привет всем любителям пончиков'])

{'NEUTRAL': 0.18066978454589844,
 'POSITIVE': 0.06783491373062134,
 'NEGATIVE': 0.7514952421188354}

In [31]:
rusentiment = pd.read_csv(DATA_PATH / 'raw' / 'sentiment' / 'rusentiment_random_posts.csv')

In [32]:
rusentiment.head(10)

,label,text
0,negative,"А попа подозревала давно,что ты с кавказа..пер..."
1,speech,З прошедшим Днем Ангела))))))))
2,skip,Два дня до отлёта с острова!!!!!!!
3,negative,"Блин, почему эта жизнь столь не справедлива (((("
4,skip,где еще встречать свой день рождения как не на...
5,neutral,Решите уравнение мужских дел для жизни :\nДомо...
6,positive,патамушта я крутая баба! ееее!!
7,neutral,Просто пост :)
8,positive,урря!я дождался этой овцыы)
9,speech,"С Днем Рождения желаю много счастья, любви и у..."


In [33]:
rusentiment.label.value_counts()

neutral     8323
positive    4635
skip        3190
speech      2826
negative    2294
Name: label, dtype: int64

In [34]:
rusentiment = rusentiment[rusentiment.label != 'speech']
rusentiment = rusentiment[rusentiment.label != 'skip']

In [35]:
rusentiment.label.value_counts()

neutral     8323
positive    4635
negative    2294
Name: label, dtype: int64

In [38]:
rutweet = pd.read_csv(DATA_PATH / 'raw' / 'sentiment' / 'ru-tweet-corp.csv', names=['text', 'label'], usecols=[4, 5])

In [39]:
rutweet.head(10)

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1
5,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1
6,"RT @SpoonLamer: Ох,900 :D ну это конечно же @t...",1
7,RT @veregijytaqo: У тебя есть ухажёр? Нет - мо...,1
8,Поприветствуем моего нового читателя @Alexey17...,1
9,Теперь у меня есть частичка Сиднея :) #Sydney ...,1


In [63]:
rutweet.label.value_counts()

1    114911
2    111923
Name: label, dtype: int64

In [62]:
rutweet['label'][rutweet['label'] == -1] = 2

C:\Users\uiqko\AppData\Local\Temp\ipykernel_31320\4024447147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rutweet['label'][rutweet['label'] == -1] = 2


In [94]:
rusentiment['label'][rusentiment['label'] == 'negative'] = 2
rusentiment['label'][rusentiment['label'] == 'positive'] = 1
rusentiment['label'][rusentiment['label'] == 'neutral'] = 0

In [107]:
kaggle_news = pd.read_json(DATA_PATH / 'raw' / 'sentiment' / 'kagglenews.json')

In [110]:
kaggle_news.head(10)

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,2
1,Медики рассказали о состоянии пострадавшего му...,1957,2
2,"Прошел почти год, как железнодорожным оператор...",1969,2
3,По итогам 12 месяцев 2016 года на территории р...,1973,2
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,2
5,Об аресте бывшего министра национальной эконом...,1980,2
6,Двое налетчиков совершили нападение на охранни...,1982,2
7,"""Самрук-Энерго"" в I квартале почти вдвое снизи...",1983,2
8,В 2016 году почти половина коррупционных прест...,1985,2
9,Google+\nЧт 19 янв 2017 12:35:02\nОдин из фигу...,1988,2


In [109]:
kaggle_news['sentiment'][kaggle_news['sentiment'] == 'negative'] = 2
kaggle_news['sentiment'][kaggle_news['sentiment'] == 'positive'] = 1
kaggle_news['sentiment'][kaggle_news['sentiment'] == 'neutral'] = 0

C:\Users\uiqko\AppData\Local\Temp\ipykernel_31320\1009553708.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_news['sentiment'][kaggle_news['sentiment'] == 'negative'] = 2
C:\Users\uiqko\AppData\Local\Temp\ipykernel_31320\1009553708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_news['sentiment'][kaggle_news['sentiment'] == 'positive'] = 1
C:\Users\uiqko\AppData\Local\Temp\ipykernel_31320\1009553708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [113]:
kaggle_news.rename(columns={'sentiment': 'label'}, inplace=True)

In [123]:
y_pred = []
y_true = []

In [124]:
dataset = shuffle(kaggle_news)
model = blanchefort
# for dataset in [rusentiment, rutweet]:
for batch in tqdm(gen_batches(len(dataset), 64), total=int(len(dataset) / 64)):
    pred = blanchefort._predict(dataset.text[batch].tolist()).tolist()

    y_pred += list(map(np.argmax, pred))
    y_true += dataset.label[batch].tolist()


  0%|          | 0/129 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [125]:
print(classification_report(y_true, y_pred, digits=2))

              precision    recall  f1-score   support

           0       0.50      1.00      0.66       694
           1       0.82      0.02      0.04       455
           2       1.00      0.02      0.03       259

    accuracy                           0.50      1408
   macro avg       0.77      0.34      0.24      1408
weighted avg       0.69      0.50      0.34      1408

